## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-10-20-54-59 +0000,nyt,Spending Bill Would Pave Way for Senators to S...,https://www.nytimes.com/2025/11/10/us/politics...
1,2025-11-10-20-50-38 +0000,nyt,Why the BBC Is Facing Its Gravest Crisis in De...,https://www.nytimes.com/2025/11/10/world/europ...
2,2025-11-10-20-46-00 +0000,wsj,Treasury Yields End Higher,https://www.wsj.com/finance/investing/jgbs-edg...
3,2025-11-10-20-45-13 +0000,missionlocal,S.F. police searching for missing 10-year-old ...,https://missionlocal.org/2025/11/sf-police-mis...
4,2025-11-10-20-42-35 +0000,nypost,"NYC mom stabbed ‘50 times,’ left to die by der...",https://nypost.com/2025/11/10/us-news/nyc-mom-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2306/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
127,trump,34
66,shutdown,30
65,government,19
204,deal,14
78,court,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
37,2025-11-10-19-39-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...,131
33,2025-11-10-19-40-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,112
173,2025-11-10-13-44-58 +0000,nypost,"Democrats melt down, call for Chuck Schumer’s ...",https://nypost.com/2025/11/10/us-news/democrat...,100
288,2025-11-09-22-28-52 +0000,nypost,Senate eyes vote on potential breakthrough to ...,https://nypost.com/2025/11/09/us-news/senate-e...,92
256,2025-11-10-03-56-00 +0000,wsj,The Senate late Sunday cleared a critical proc...,https://www.wsj.com/politics/policy/republican...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
37,131,2025-11-10-19-39-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...
158,66,2025-11-10-14-58-51 +0000,nypost,Supreme Court rejects Kim Davis bid to overtur...,https://nypost.com/2025/11/10/us-news/supreme-...
33,63,2025-11-10-19-40-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...
170,50,2025-11-10-14-01-26 +0000,nypost,Ex-French President Nicolas Sarkozy to be rele...,https://nypost.com/2025/11/10/world-news/franc...
156,46,2025-11-10-15-03-47 +0000,cbc,8 people dead in car explosion near Red Fort i...,https://www.cbc.ca/news/world/india-explosion-...
144,45,2025-11-10-15-58-48 +0000,nypost,NYC realtor fired over controversial Zohran Ma...,https://nypost.com/2025/11/10/business/nyc-rea...
132,45,2025-11-10-16-35-00 +0000,wsj,Sen. Bernie Sanders and a group of Democratic ...,https://www.wsj.com/politics/policy/the-politi...
81,41,2025-11-10-18-11-45 +0000,wapo,Super Typhoon Fung-wong slams Philippines in w...,https://www.washingtonpost.com/world/2025/11/0...
186,34,2025-11-10-12-30-00 +0000,wsj,Private-Label Food Maker TreeHouse to Go Priva...,https://www.wsj.com/business/retail/private-la...
201,29,2025-11-10-11-40-11 +0000,nypost,Trump threatens BBC with $1B lawsuit over doct...,https://nypost.com/2025/11/10/us-news/trump-la...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
